In [5]:
%matplotlib inline
import numpy as np
import nidaqmx as ni
import matplotlib.pyplot as plt
import time

## Set up DAQ

In [6]:
system = ni.system.System.local()
device = system.devices['Dev1']

The following cell initialize the DAQ device for acepting the external io. Do this only once when the DAQ is on.

In [10]:
if 0:
    write_trigger.close()
    write_voltage.close()
    read_done.close()

write_trigger=ni.task.Task(new_task_name='task1')
write_trigger.do_channels.add_do_chan('Dev1/port0/line0')
write_voltage=ni.task.Task(new_task_name='task2')
write_voltage.ao_channels.add_ao_voltage_chan('Dev1/ao0:1', max_val=10, min_val=-10)
read_done=ni.task.Task(new_task_name='task3')
read_done.di_channels.add_di_chan('Dev1/port0/line1')

DIChannel(name=Dev1/port0/line1)

In [11]:
write_trigger.start()
read_done.start()
write_voltage.start()

## Create voltage list

In [12]:
xy_cent=np.array([0.525,-0.046]) #volts
volt_per_deg=0.5
max_NA=0.35
r_led=7 #radius, in unit of "# of leds"

[NA_x,NA_y]=np.meshgrid(np.linspace(-max_NA,max_NA,2*r_led+1),np.linspace(-max_NA,max_NA,2*r_led+1))
NA_x[np.arange(1,NA_x.shape[0],2),:]=NA_x[np.arange(1,NA_x.shape[0],2),::-1] #Flip every other column
#Limit down to a circle
valid_led=(NA_x**2+NA_y**2)<=(max_NA**2)

NA_x=NA_x[valid_led]
NA_y=NA_y[valid_led]

angleD_x=np.arcsin(NA_x)*180/np.pi
angleD_y=np.arcsin(NA_y)*180/np.pi

v_x=angleD_x*volt_per_deg/2 + xy_cent[0]
v_y=angleD_y*volt_per_deg/2 + xy_cent[1]

num_galvo=len(v_x)

v_xx=v_x
v_yy=v_y

In [13]:
num_galvo

149

In [34]:
xy_cent=np.array([0.525,-0.046]) #volts
write_voltage.write(xy_cent)

1

In [18]:
xy_cent=np.array([2, 0.242]) #volts
write_voltage.write(xy_cent)

1

In [ ]:
n_img = 350
v_x = np.ones(n_img)*xy_cent[0]
v_y =np.ones(n_img)*xy_cent[1]
num_galvo = n_img

## Save it
num_galvo

In [ ]:
write_voltage.write(np.array([1,1])*2)
write_trigger.write(False)

In [ ]:
# Vary y
temp=np.arange(-3,3,0.1)
v_y=np.concatenate((np.arange(0,3,0.1),np.arange(3,-3,-0.1),np.arange(-3,0,0.1)))+xy_cent[1]
v_x=np.ones(v_y.shape,float)*xy_cent[0]
num_galvo=len(v_x)


In [6]:
# Vary x
temp=np.arange(-3,3,0.1)
v_x=np.concatenate((np.arange(0,3,0.1),np.arange(3,-3,-0.1),np.arange(-3,0,0.1)))+xy_cent[0]
v_y=np.ones(v_x.shape,float)*xy_cent[1]
num_galvo=len(v_x)

In [7]:
write_voltage.write(xy_cent)
#for vv in range(num_galvo-1):
try:
    vv=0
    while True:
        vv=vv+1
        if vv>=num_galvo:
            vv=0

        success=write_voltage.write(np.array([v_x[vv],v_y[vv]])) #Write voltage
        if success == 0:
            print('Failed to write, stopping.')
            break
        time.sleep(0.01)
except KeyboardInterrupt:
    pass

DaqError: The specified operation cannot be performed because a task has been aborted or a device has been removed from the system. Handle this situation as required by the application and then, if appropriate, attempt to perform the operation again.
Task Name: task2

Status Code: -88709

In [ ]:
maxNA=0.38
voltage=np.arcsin(np.array([-maxNA, maxNA]))*(180/np.pi)*volt_per_deg/2
xy_mnmx=np.concatenate((voltage+xy_cent[0],voltage+xy_cent[1]))


In [ ]:
write_voltage.write(np.array([0,xy_cent[1]]))

In [ ]:
na=np.sin(np.array([-5.6-xy_cent[0], 6.4-xy_cent[0], -7.4-xy_cent[1], 7.6-xy_cent[1]])*(2/volt_per_deg)*(np.pi/180))
print(na)

##### Galvo control

In [14]:
#Set back to 0,0
write_voltage.write(xy_cent)
write_trigger.write(False)
counter_cur=0
counter_val=0

for vv in range(num_galvo):
    print('Waiting for camera signal.')
    while int(counter_cur) <=int(counter_val):
        success=read_done.read()
        if success==False:
            counter_cur=counter_val+1
        time.sleep(0.005)
            
    print('Writing voltage to galvo.')
    counter_val=counter_cur #Replace value when incremented
    success=write_voltage.write(np.array([v_x[vv],v_y[vv]])) #Write voltage
    if success == 0:
        print('Failed to write, stopping.')
        break
    print('Sending camera trigger signal.')
    #time.sleep(5)
    success=write_trigger.write(True)
    if success == 0:
        print('Failed to write, stopping.')
        break
    time.sleep(0.005)
    success=write_trigger.write(False)
    if success == 0:
        print('Failed to write, stopping.')
        break
    print('Waiting for camera signal.')
    
while int(counter_cur) <=int(counter_val):
    success=read_done.read()
    if success==False:
        counter_cur=counter_val+1
    time.sleep(0.005)
write_voltage.write(xy_cent)

Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
W

Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
Writing voltage to galvo.
Sending camera trigger signal.
Waiting for camera signal.
Waiting for camera signal.
W

1

In [35]:
write_trigger.stop()
write_voltage.stop()
read_done.stop()

write_trigger.close()
write_voltage.close()
read_done.close()
#read_test.close()

In [8]:
write_trigger.close()

In [9]:
write_trigger.close()
write_voltage.close()
read_done.close()

C:\Users\Grant\Anaconda3\lib\site-packages\nidaqmx\task.py:463: ResourceWarning: Attempted to close NI-DAQmx task of name "task1" but task was already closed.
  'already closed.'.format(self._saved_name), DaqResourceWarning)
